# File Management Functions

In [1]:
import os
import json
from urllib.parse import urlsplit
import shutil

def openModule(folder, pattern='^day\d\d.*$'):
    import re
    
    all_files = os.listdir(folder)
    
    files = [file for file in all_files if re.match(pattern, file)]
    
    files.sort()

    days = []
    
    for file in files:
        # print(file)
        try:
            day = json.load(fp=open(f'{folder}/{file}','r'))
            days.append(day)
        
        except Exception as e:
            print('skipping:', f'{folder}/{file}', e )
            files.remove(file)
    
    return days, files

def getFileList(items):

    if "items" not in items:
        return []    
    
    items = items['items']
    links = [item['url'] for item in items]

    # if len(links) == 0: return []

    return links

class DayFileList:
    def __init__(self, day):
        self.preClass = getFileList(day['dateTypes']['general']['preClass'])
        self.preClass += getFileList(day['dateTypes']['css']['preClass'])
        self.inClass = getFileList(day['dateTypes']['general']['inClass'])
        self.inClass += getFileList(day['dateTypes']['css']['inClass'])
        self.postClass = getFileList(day['dateTypes']['general']['postClass'])
        self.postClass += getFileList(day['dateTypes']['projects']['postClass'])


class AlgoDayFileList:
    def __init__(self, day):
        self.preClass = getFileList(day['preClass'])
        self.inClass = getFileList(day['inClass'])
        self.postClass = getFileList(day['postClass'])




def copyFile(file, dest=''):
    basepath = './old_bootcamp_docs'
    
    components = urlsplit(file)
    filepath = os.path.split(components.path)[0]+'.md'
    filename = dest + '/' + os.path.split(components.path)[1]+'.md'
    # print(components.path)
    # print(filepath)
    try :
        src = basepath+components.path+'.md'
        dest = filename
        # print('copy', src, 'to', dest)
        shutil.copyfile(src, dest)
    except :
        
        # print('get readme')
        src = basepath+components.path+'/README.md'
        dest = filename
        # print('copy', src, 'to', dest)
        try :
            shutil.copyfile(src, dest)       
        except:
            print('file not found:', src, 'cannot copy to', dest)


def constructDayFolder(day, dayNo, folder):

    # dayNo = 1
    dayPath = f'{folder}/day{dayNo}'
    preclassFolder = f'{dayPath}/preclass'
    inclassFolder = f'{dayPath}/inclass'
    postclassFolder = f'{dayPath}/postclass'

    
    try :
        os.mkdir(dayPath)    
        os.mkdir(preclassFolder)
        os.mkdir(inclassFolder)
        os.mkdir(postclassFolder)
    except:
        pass #print('folder exists')
      
    for file in day.preClass:
        copyFile(file, dest=preclassFolder)
    for file in day.inClass:
        copyFile(file, dest=inclassFolder)
    for file in day.postClass:
        copyFile(file, dest=postclassFolder)



def constructAlgosDayFolder(day, dayNo, folder='algos'):

    # dayNo = 1
    dayPath = f'{folder}/day{dayNo}'
    
    try :
        os.mkdir(dayPath)    

    except:
        pass # print('folder exists')
      
    for file in day.preClass:
        copyFile(file, dest=dayPath)
    for file in day.inClass:
        copyFile(file, dest=dayPath)
    for file in day.postClass:
        copyFile(file, dest=dayPath)    

def reconstructModule(folder, pattern='^day\d\d.*$'): 
    days,_ = openModule(folder, pattern)

    for i in range(len(days)):
        day = DayFileList(days[i])
        constructDayFolder(day,i+1,folder)


# Summary Generation Functions 


module_longnames = {
  'Module1' : 'Module 1: Basic Frontend',
  'Module2' : 'Module 2: Basic Backend',
  'Module3' : 'Module 3: Advanced Backend and Full Stack',
  'Module4' : 'Module 4: Frontend Framework - React',
  'Module5' : 'Module 5: Advanced React',
  'Module6' : 'Module 6: Capstone Project',  # Branching out to become a Software Engineer

}


def processItems(items):
    if "items" not in items:
        return {'links':[], 'names':[]}   
    
    items = items['items']
    names = [item['name'] for item in items]
    links = [item['url'] for item in items]

    # if len(links) == 0: return []

    return {'links':links, 'names':names} 

def getItems(items):
    try: 
        return items['items']
    except:
        return []
    

def getNewPath(url,dayNo,folder, dest):
        from urllib.parse import urlsplit
        components = urlsplit(url)
        
        filename = dest + '/' + os.path.split(components.path)[1]+'.md'

        return filename
    

def getModuleSummary(folder, pattern='^day\d\d.*.json$'): 
    days,files = openModule(folder, pattern)
    def output(s):
        # print(s)
        summary_file = open('summary2.md', 'a')
        summary_file.write(s+'\n')
        summary_file.close()


    output(f"\n## {module_longnames[folder]}")
    output(f"\n- {module_longnames[folder]}")

    for i in range(len(days)):

        dayPath = f'{folder}/day{i+1}'
        preclassFolder = f'{dayPath}/preclass'
        inclassFolder = f'{dayPath}/inclass'
        postclassFolder = f'{dayPath}/postclass'

        day = days[i]
        output(f'  - Day {i+1}')
        output(f'    - Pre Class')
        
        for item in getItems(day['dateTypes']['general']['preClass']):
            name = item['name']
            link = getNewPath(item['url'], i+1, folder, preclassFolder)
            output(f'      * [{name}]({link})')
        for item in getItems(day['dateTypes']['css']['preClass']):
            name = item['name']
            link = getNewPath(item['url'], i+1, folder, preclassFolder)
            output(f'      * [{name}]({link})')
        output('    - In Class')
        for item in getItems(day['dateTypes']['general']['inClass']):
            name = item['name']
            link = getNewPath(item['url'], i+1, folder, inclassFolder)
            output(f'      * [{name}]({link})')
        for item in getItems(day['dateTypes']['css']['inClass']):
            name = item['name']
            link = getNewPath(item['url'], i+1, folder, inclassFolder)
            output(f'      * [{name}]({link})')
        output('    - Post Class')
        for item in getItems(day['dateTypes']['general']['postClass']):
            name = item['name']
            link = getNewPath(item['url'], i+1, folder, postclassFolder)
            output(f'      * [{name}]({link})')
        for item in getItems(day['dateTypes']['projects']['postClass']):
            name = item['name']
            link = getNewPath(item['url'], i+1, folder, postclassFolder)
            output(f'      * [{name}]({link})')

    
# Table Generation Functions

from IPython.display import HTML, display



def processItemsForDisplay(items,linkPath):
    if "items" not in items:
        return '' #None
    
    
    items = items['items']
    names = [item['name'] for item in items]
    links = [item['url'] for item in items]
    if len(names) == 0: return ''#None

    output = ''
    for i in range(len(names)):
        name = names[i]
        a,b = 0,0
        link = getNewPath(links[i],a,b, linkPath)
        output = output + f'<li>[{name}]({link})</li>'        

    return output
    

    # if items is None or len(items) == 0 : 
    #     return 'None'
    # else:
    #     return [item['name'] for item in items]

def generateOutline(days):

    day_number = ['day#']
    preclass = ['preclass']
    inclass = ['inclass']
    postclass = ['postclass']

    i=1
    for day in days:
        day_number.append(i)
        i += 1
        preclass.append(processItemsForDisplay(day['dateTypes']['general']['preClass']))
        inclass.append(processItemsForDisplay(day['dateTypes']['general']['inClass']))
        postclass.append([processItemsForDisplay(day['dateTypes']['general']['postClass']),processItemsForDisplay(day['dateTypes']['projects']['postClass'])])
        

    return [day_number,preclass, inclass, postclass]


def generateOutlineDayRows(days, moduleFolder):

    outline = []
    outline.append(['Day No.','Pre-Class', 'In-Class', 'Post-Class'])

    i=1
    for day in days:
        row = []

        dayPath = f'day{i}'
        preclassFolder = f'{dayPath}/preclass'
        inclassFolder = f'{dayPath}/inclass'
        postclassFolder = f'{dayPath}/postclass'
        
        row.append(f'<h2>Day {i}</h2>')
        i += 1

        
        linkPath = preclassFolder 
        row.append(processItemsForDisplay(day['dateTypes']['general']['preClass'],linkPath) + '<br>' + processItemsForDisplay(day['dateTypes']['css']['preClass'],linkPath))
        # processItems(day['dateTypes']['css']['inClass'])]
        linkPath = inclassFolder
        row.append(processItemsForDisplay(day['dateTypes']['general']['inClass'],linkPath) + '<br>' + processItemsForDisplay(day['dateTypes']['css']['inClass'],linkPath))
        
        linkPath = postclassFolder
        allPostClass = ''
        for x in [processItemsForDisplay(day['dateTypes']['general']['postClass'],linkPath), processItemsForDisplay(day['dateTypes']['projects']['postClass'],linkPath)]:
            if x : allPostClass += x + '<br>'
         
         
        row.append(allPostClass)

        outline.append(row)
        

    return outline

def displayTableFromData(data):
    display(HTML(
    '<table><tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
        )
    ))

def outputTableFromData(data):
    
    out = (
    '<table><tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
        )
    )
    # print(out)
    return out



def displayModuleTable(days):

    # module_outline = generateOutline(days)
    module_outline = generateOutlineDayRows(days)

    data = module_outline #[list(x) for x in zip(module_outline)]

    displayTableFromData(data)

def generateOutlineAlgos(days):

    outline = []
    outline.append(['Day No.','Pre-Class', 'In-Class', 'Post-Class'])

    i=1
    for day in days:
        row = []

        row.append(i)
        i += 1
        row.append(processItemsForDisplay(day['preClass']))
        row.append(processItemsForDisplay(day['inClass']))
        row.append(processItemsForDisplay(day['postClass']))


        outline.append(row)
        

    return outline


def displayAlgosTable(algosJson):
    data = generateOutlineAlgos(algosJson)
    displayTableFromData(data)


# Markdown Table Generation function

def outputTableMDFromData(data):
    
    out = '|Day No.|Pre-Class|In-Class|Post-Class|\n'
    out += '| --- | --- | --- | --- |\n'

    data.pop(0)

    for row in data:
        rowMd = '|'
        for item in row:
            rowMd += f'{item}|'
        rowMd += '\n' 
        out += rowMd

    # out = (
    
    #     ''.join(
    #         '|{}|\n'.format('|'.join(str(_) for _ in row)) for row in data) 
    #     )
    
    # print(out)
    return out



# Execute File Manaagement to create Folder Structure

In [12]:
folder = 'Module4'
print(folder)
reconstructModule('Module4')

Module1
skipping: Module1/day10 [Errno 21] Is a directory: 'Module1/day10'
skipping: Module1/day11 [Errno 21] Is a directory: 'Module1/day11'
skipping: Module1/day12 [Errno 21] Is a directory: 'Module1/day12'
skipping: Module1/day13 [Errno 21] Is a directory: 'Module1/day13'
skipping: Module1/day14 [Errno 21] Is a directory: 'Module1/day14'
skipping: Module1/day15 [Errno 21] Is a directory: 'Module1/day15'


In [2]:


folder = 'Module1'
print(folder)
reconstructModule('Module1')


print('Module2')
patternStartsWithNumber = "^\d{1,2}-.*$"
reconstructModule('Module2', patternStartsWithNumber)

for i in [3,4,5,6]:
    moduleString = f'Module{i}'
    print(moduleString)
    reconstructModule(moduleString)


algos = json.load(fp=open('algos/algos.json','r'))
days = algos

for i in range(len(days)):
    day = AlgoDayFileList(days[i])
    constructAlgosDayFolder(day,i+1,'algos')

Module1
skipping: Module1/day10 [Errno 21] Is a directory: 'Module1/day10'
skipping: Module1/day11 [Errno 21] Is a directory: 'Module1/day11'
skipping: Module1/day12 [Errno 21] Is a directory: 'Module1/day12'
skipping: Module1/day13 [Errno 21] Is a directory: 'Module1/day13'
skipping: Module1/day14 [Errno 21] Is a directory: 'Module1/day14'
skipping: Module1/day15 [Errno 21] Is a directory: 'Module1/day15'
Module2
Module3
skipping: Module3/day10 [Errno 21] Is a directory: 'Module3/day10'
skipping: Module3/day12 [Errno 21] Is a directory: 'Module3/day12'
skipping: Module3/day14 [Errno 21] Is a directory: 'Module3/day14'
skipping: Module3/day16 [Errno 21] Is a directory: 'Module3/day16'
Module4
skipping: Module4/day10 [Errno 21] Is a directory: 'Module4/day10'
skipping: Module4/day12 [Errno 21] Is a directory: 'Module4/day12'
skipping: Module4/day14 [Errno 21] Is a directory: 'Module4/day14'
skipping: Module4/day16 [Errno 21] Is a directory: 'Module4/day16'
Module5
Module6
skipping: Modu

# Execute Summary Generation for Modules 1-6

In [20]:

# reset the file to be empty
summary_file = open('summary2.md', 'w')
summary_file.close()


getModuleSummary("Module1")
    
patternStartsWithNumber = "^\d{1,2}-.*$"
getModuleSummary('Module2', patternStartsWithNumber)

for i in [3,4,5,6]:
    moduleString = f'Module{i}'
    print(moduleString)
    getModuleSummary(moduleString)




Module3
Module4
Module5
Module6


In [34]:
getModuleSummary('Module3')
# folder = 'Module3'
# days,files = openModule(folder)
# print(files)



# Execute Summary Generation for Algos

In [23]:
algos = json.load(fp=open('algos/algos.json','r'))
print('algos')

def output(s):
    # print(s)
    summary_file = open('summary2.md', 'a')
    summary_file.write(s+'\n')
    summary_file.close()



output(f"# Algorithms")
output(f"- Algorithms")

for i in range(len(algos)):
    folder = 'algos'
    dayPath = f'{folder}/day{i+1}'

    day = algos[i]
    output(f'  - Day {i+1}')
    # output(f'    - Pre Class')
    
    for item in getItems(day['preClass']):
        name = item['name']
        link = getNewPath(item['url'], i+1, folder, dayPath)
        output(f'    * [{name}]({link})')
    
    # output('    - In Class')
    for item in getItems(day['inClass']):
        name = item['name']
        link = getNewPath(item['url'], i+1, folder, dayPath)
        output(f'    * [{name}]({link})')
   
    # output('    - Post Class')
    for item in getItems(day['postClass']):
        name = item['name']
        link = getNewPath(item['url'], i+1, folder, dayPath)
        output(f'    * [{name}]({link})')
    

algos


# Execute HTML Table Generation for Modules

In [15]:
days_m1, module1_files = openModule("Module1",pattern='^day\d\d.json')
# displayModuleTable(days_m1)
html_output = outputTableFromData(generateOutlineDayRows(days_m1,moduleFolder="Module1"))

summary_file = open('Module1/README.md', 'w')
summary_file.write(html_output+'\n')
summary_file.close()


patternStartsWithNumber = "^\d{1,2}-.*$"
days_m2, module2_files = openModule("Module2",pattern=patternStartsWithNumber)

html_output = outputTableFromData(generateOutlineDayRows(days_m2, moduleFolder="Module2"))

summary_file = open('Module2/README.md', 'w')
summary_file.write(html_output+'\n')
summary_file.close()


for i in [3,4,5,6]:
    days, module_files = openModule(f"Module{i}",pattern='^day\d\d.json')
    # displayModuleTable(days_m1)
    html_output = outputTableFromData(generateOutlineDayRows(days, moduleFolder=f'Module{i}'))

    summary_file = open(f'Module{i}/README.md', 'w')
    summary_file.write(html_output+'\n')
    summary_file.close()

# markdown table generation

In [6]:
days_m1, module1_files = openModule("Module1",pattern='^day\d\d.json')
day_rows = generateOutlineDayRows(days_m1,moduleFolder="Module1")
html_output = outputTableMDFromData(day_rows)

summary_file = open('Module1/README.md', 'w')
summary_file.write(html_output+'\n')
summary_file.close()

In [21]:


days_m1, module1_files = openModule("Module1",pattern='^day\d\d.json')
day_rows = generateOutlineDayRows(days_m1,moduleFolder="Module1")
html_output = outputTableMDFromData(day_rows)

summary_file = open('Module1/README.md', 'w')
summary_file.write(html_output+'\n')
summary_file.close()

patternStartsWithNumber = "^\d{1,2}-.*$"
days_m2, module2_files = openModule("Module2",pattern=patternStartsWithNumber)

html_output = outputTableMDFromData(generateOutlineDayRows(days_m2, moduleFolder="Module2"))

summary_file = open('Module2/README.md', 'w')
summary_file.write(html_output+'\n')
summary_file.close()


for i in [3,4,5,6]:
    days, module_files = openModule(f"Module{i}",pattern='^day\d\d.*')
    html_output = outputTableMDFromData(generateOutlineDayRows(days, moduleFolder=f'Module{i}'))

    summary_file = open(f'Module{i}/README.md', 'w')
    summary_file.write(html_output+'\n')
    summary_file.close()

skipping: Module3/day10 [Errno 21] Is a directory: 'Module3/day10'
skipping: Module3/day12 [Errno 21] Is a directory: 'Module3/day12'
skipping: Module3/day14 [Errno 21] Is a directory: 'Module3/day14'
skipping: Module3/day16 [Errno 21] Is a directory: 'Module3/day16'
skipping: Module4/day10 [Errno 21] Is a directory: 'Module4/day10'
skipping: Module4/day12 [Errno 21] Is a directory: 'Module4/day12'
skipping: Module4/day14 [Errno 21] Is a directory: 'Module4/day14'
skipping: Module4/day16 [Errno 21] Is a directory: 'Module4/day16'
skipping: Module6/day10 [Errno 21] Is a directory: 'Module6/day10'


In [19]:
days_m1, module1_files = openModule("Module1",pattern='^day\d\d.json')
day_rows = generateOutlineDayRows(days_m1,moduleFolder="Module1")
html_output = outputTableMDFromData(day_rows)

summary_file = open('Module1/README.md', 'w')
summary_file.write(html_output+'\n')
summary_file.close()

#  Scheduler generation

In [24]:
empty_schedule_json = '''{
  "repoUrls": {
    "edit": "https://github.com/rocketacademy/scheduler/edit/main/src/data/bootcamp-course-days.json"
  },
  "daysOfWeek": [1, 2, 3, 4],
  "courseStartIndex": 0,
  "totalCourseDays": 116,
  "days": [
    
    
  ]
}'''


constructed_schedule = json.loads(empty_schedule_json)
day0 = json.loads('''{
      "courseDay": 0,
      "dateTypes": {
        "module": "Module 1: Frontend Basics",
        "general": {
          "type": "general",
          "preClass": {},
          "inClass": {
            "items": [
              {
                "name": "Day 1 Photo!",
                "url": ""
              },
              {
                "name": "Course and Student Introductions",
                "url": ""
              }
            ]
          },
          "postClass": {}
        }
      }
    }''')
days = []
days.append(day0)
constructed_schedule['days'] = days
# print(daysm1[0])
# print(filesm1)


deployed_urls = {
  'Module1': 'https://swebootcamp.rocketacademy.co/module-1-basic-frontend/module1',
  'Module2': 'https://swebootcamp.rocketacademy.co/module-2-basic-backend/module2',
  'Module3': 'https://swebootcamp.rocketacademy.co/module-3-advanced-backend-and-full-stack/module3',
  'Module4': 'https://swebootcamp.rocketacademy.co/module-4-frontend-framework-react/module4',
  'Module5': 'https://swebootcamp.rocketacademy.co/module-5-advanced-react/module-5-advanced-react',
  'Module6': 'https://swebootcamp.rocketacademy.co/module-6-branching-out-to-become-a-software-engineer/module6',
}

patternStartsWithNumber = "^\d{1,2}-.*$"
file_regex_patterns = {
  'Module1' : '^day\d\d.*$',
  'Module2' : patternStartsWithNumber,
  'Module3' : '^day\d\d.*$',
  'Module4' : '^day\d\d.*$',
  'Module5' : '^day\d\d.*$',
  'Module6' : '^day\d\d.*$',  

}


# folder = 'Module1'
# daysm1, filesm1 = openModule(folder)
courseDay = 0
for folder, pattern in file_regex_patterns.items():
    module_days, files = openModule(folder, pattern)
    for i in range(len(files)):
        file = files[i]
        courseDay += 1
        dayEntry = {
          "courseDay": courseDay,
          "dateTypes": {
            "module": module_longnames[folder],
            "general": {
              "type": "general",
              "preClass": {},
              "inClass": {
                "items": [
                  {
                    "name": f"Day {i+1} Coursework",
                    "url": f"{deployed_urls[folder]}/day-{i+1}"
                  }
                ]
              },
              "postClass": {}
            }
          }
        }
        days.append(dayEntry)
        print(dayEntry)

constructed_schedule['totalCourseDays'] = courseDay
constructed_schedule['days'] = days

json.dump(constructed_schedule,fp=open('bc2.0_days.json','w'))

skipping: Module1/day10 [Errno 21] Is a directory: 'Module1/day10'
skipping: Module1/day11 [Errno 21] Is a directory: 'Module1/day11'
skipping: Module1/day12 [Errno 21] Is a directory: 'Module1/day12'
skipping: Module1/day13 [Errno 21] Is a directory: 'Module1/day13'
skipping: Module1/day14 [Errno 21] Is a directory: 'Module1/day14'
skipping: Module1/day15 [Errno 21] Is a directory: 'Module1/day15'
{'courseDay': 1, 'dateTypes': {'module': 'Module 1: Basic Frontend', 'general': {'type': 'general', 'preClass': {}, 'inClass': {'items': [{'name': 'Day 1 Coursework', 'url': 'https://swebootcamp.rocketacademy.co/module-1-basic-frontend/module1/day-1'}]}, 'postClass': {}}}}
{'courseDay': 2, 'dateTypes': {'module': 'Module 1: Basic Frontend', 'general': {'type': 'general', 'preClass': {}, 'inClass': {'items': [{'name': 'Day 2 Coursework', 'url': 'https://swebootcamp.rocketacademy.co/module-1-basic-frontend/module1/day-2'}]}, 'postClass': {}}}}
{'courseDay': 3, 'dateTypes': {'module': 'Module 1